In [ ]:
#1 corr xtalk from I, using continuum means
mean = np.mean(dat[:,:,:,:], axis=(0,1))
print(mean.shape)

#I-V
a = np.mean(mean[3][100:230])
print(a,'=a')
b1 = np.mean(mean[1][100:230])
print(b1,'=b1')
g1 = np.mean(mean[2][100:230])
print(g1,'=g1')

#def new stk
stk_new = np.zeros(dat.shape)
stk_new[:,:,0,:] = dat[:,:,0,:]
stk_new[:,:,1,:] = dat[:,:,1,:] - b1*dat[:,:,0,:]
stk_new[:,:,2,:] = dat[:,:,2,:] - g1*dat[:,:,0,:]
stk_new[:,:,3,:] = dat[:,:,3,:] - a*dat[:,:,0,:]

print(stk_new.shape, 'new stk')

plt.figure(figsize=(12,2))
plt.subplot(121)
plt.plot(np.mean(stk_new[:,:,1,:],axis=(0,1)))
plt.plot(np.mean(stk_new[:,:,2,:],axis=(0,1)))
plt.plot(np.mean(stk_new[:,:,3,:],axis=(0,1)))
plt.grid()
plt.title('crosstalk I. [Q,U,V - params * I]')
plt.subplot(122)
plt.plot(mean[1])
plt.plot(mean[2])
plt.plot(mean[3])
plt.grid()
plt.title('before')

In [ ]:
#use sec L from paper. follow paper jaeggli exact
from scipy.optimize import minimize

def L_paper(par,  stk):   #par = parameters which will be minimized
    b2 = par[0]
    g2 = par[1]
    Q = stk[:,:,1,:] - b2*stk[:,:,3,:]
    U = stk[:,:,2,:] - g2*stk[:,:,3,:]
    V = stk[:,:,3,:]   #input stk param after xtalk I removal
    
    S = np.abs(np.sum(Q*V, axis=2)) + np.abs(np.sum(U*V, axis=2)) + np.sum(V, axis=2)**2 #sum over wavelens
    
    out = np.sum(S, axis=(0,1)) # spatial sum
    return out  

xtalk2 = minimize(L_paper, x0=(-0.01,-0.01) , args=stk_new)

print(xtalk2.x[0], 'b2')
print(xtalk2.x[1], 'g2')

Q_corr = dat[:,:,1,:] - b1*dat[:,:,0,:] - xtalk2.x[0]*stk_new[:,:,3,:] 
U_corr = dat[:,:,2,:] - g1*dat[:,:,0,:] - xtalk2.x[1]*stk_new[:,:,3,:] 

plt.figure(figsize=(12,2))
plt.subplot(131)
plt.plot(np.mean(Q_corr[:,:,:], axis=(0,1)))
plt.plot(np.mean(U_corr[:,:,:], axis=(0,1)))
plt.plot(mean[3]-a)
plt.grid()
plt.title('1fixed crosstalk from I and V')
plt.subplot(132)
plt.plot(np.mean(stk_new[:,:,1,:],axis=(0,1)))
plt.plot(np.mean(stk_new[:,:,2,:],axis=(0,1)))
plt.plot(np.mean(stk_new[:,:,3,:],axis=(0,1)))
plt.grid()
plt.title('2crosstalk I. [Q,U,V - params * I]')
plt.subplot(133)
plt.plot(mean[1])
plt.plot(mean[2])
plt.plot(mean[3])
plt.grid()
plt.title('before')